In [1211]:
import math
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans
from pulp import *

In [1212]:
img1 = Image.open("astre_bleu2.png")
img2 = Image.open("astre_rouge1.png")

In [1213]:
img1_array = np.array(img1)
img2_array = np.array(img2)

In [1214]:
print(img1_array.shape)
print(img1_array.size)
print(img1_array[0][0])
print(img1_array[0][0][0])

(300, 300, 3)
270000
[3 3 6]
3


In [1215]:
pixel_matrix = img1_array.reshape(-1, 3)
pixel_matrix2 = img2_array.reshape(-1, 3)

In [1216]:
number_cluster = 10
kmeans1 = KMeans(n_clusters=number_cluster, random_state=0).fit(pixel_matrix)
kmeans2 = KMeans(n_clusters=number_cluster, random_state=0).fit(pixel_matrix2)

In [1217]:
new_img_array = np.zeros_like(pixel_matrix)
for i, label in enumerate(kmeans1.labels_):
    new_img_array[i] = kmeans1.cluster_centers_[label]
new_img = new_img_array.reshape(img1_array.shape)

new_img_array2 = np.zeros_like(pixel_matrix2)
for i, label in enumerate(kmeans2.labels_):
    new_img_array2[i] = kmeans2.cluster_centers_[label]
new_img2 = new_img_array2.reshape(img2_array.shape)

In [1218]:
Image.fromarray(new_img).show()
Image.fromarray(new_img).save('clustered1_' + str(number_cluster) + 'clust.jpg')

In [1219]:
Image.fromarray(new_img2).show()
Image.fromarray(new_img2).save('clustered2_' + str(number_cluster) + 'clust.jpg')

In [1220]:
""" 1ère étape : Définir notre matrice de coût avec les distances euclidiennes entre les clusters_centers_ 
    2ème étape : Définir les masses de chacun des clusters en utilisant np.count_nonzero(kmeans1.labels_ == i) pour i allant de 0 à kmeans1.cluster_centers_.shape[0]
    qu'on divisera, si nécessaire, par le nombre total de pixel de l'image (qu'on obtient par exemple avec img_array.size // 3) pour construire les arrays des masses des deux images
    3ème étape : On résout le problème de transport optimal avec Pulp
    4ème étape : Avec le plan de transport/matrice de couplage, pour l'instant, on implémente une fonction qui à partir de la deuxième image effectue une permutation de ses pixels.
    Plus précisément, on itére sur les tous chemins/routes possibles, et on pour chaque chemin, i.e pour le cluster i de la première image vers le cluster j de la deuxième,
    on regarde le nombre de pixels à transporter et on les transporte.
"""

" 1ère étape : Définir notre matrice de coût avec les distances euclidiennes entre les clusters_centers_ \n    2ème étape : Définir les masses de chacun des clusters en utilisant np.count_nonzero(kmeans1.labels_ == i) pour i allant de 0 à kmeans1.cluster_centers_.shape[0]\n    qu'on divisera, si nécessaire, par le nombre total de pixel de l'image (qu'on obtient par exemple avec img_array.size // 3) pour construire les arrays des masses des deux images\n    3ème étape : On résout le problème de transport optimal avec Pulp\n    4ème étape : Avec le plan de transport/matrice de couplage, pour l'instant, on implémente une fonction qui à partir de la deuxième image effectue une permutation de ses pixels.\n    Plus précisément, on itére sur les tous chemins/routes possibles, et on pour chaque chemin, i.e pour le cluster i de la première image vers le cluster j de la deuxième,\n    on regarde le nombre de pixels à transporter et on les transporte.\n"

In [1221]:
def get_cost_matrix(cluster_start, cluster_arrival):

    height, width = cluster_start.shape[0], cluster_arrival.shape[0]
    
    res = np.zeros((height, width), dtype=np.float16)

    for i in range(height):
        for j in range(width):
            res[i, j] = math.dist(cluster_start[i], cluster_arrival[j])
    return res


In [1222]:
test = np.zeros((2, 3), dtype=np.float16)

In [1223]:
cost_matrix = get_cost_matrix(kmeans1.cluster_centers_, kmeans2.cluster_centers_)

In [1224]:
m_start = [np.count_nonzero(kmeans1.labels_ == i) for i in range(kmeans1.cluster_centers_.shape[0])]
m_arrival = [np.count_nonzero(kmeans2.labels_ == i) for i in range(kmeans2.cluster_centers_.shape[0])]

In [1225]:
def optimal_solve(costs, m_start, m_arrival):
  # Initialise les variables du probleme
  problem = LpProblem("Optimal_transport", LpMinimize)
  paths = [(i, j) for i in range(len(m_start)) for j in range(len(m_arrival))]
  path_variables = LpVariable.dicts("Path", (range(len(m_start)), range(len(m_arrival))),
                                lowBound=0, cat='Integer')

  # Definit la fonction objective
  problem += lpSum([path_variables[i][j] * costs[i][j] for (i, j) in paths])

  # Pose les contraintes
  for i in range(len(m_start)):
      problem += lpSum([path_variables[i][j] for j in range(len(m_arrival))]) == m_start[i]
  for j in range(len(m_arrival)):
      problem += lpSum([path_variables[i][j] for i in range(len(m_start))]) == m_arrival[j]
  # Utilise la méthode solve() associee à l'objet de type LpProblem pour resoudre le problème de programmation lineaire  
  problem.solve()

  # Stocke les resultats dans un dictionnaire
  result = {}
  for v in problem.variables():
      route = v.name.split("_")[1:]
      i, j = int(route[0]), int(route[1])
      result[i, j] = v.varValue
  # Construit la matrice de couplage / le plan de transport optimal  
  optimal_plan = np.zeros((len(m_start), len(m_arrival)))
  for i, j in result.keys():
    optimal_plan[i,j] = result[i, j]
  return optimal_plan


In [1226]:
optimal_plan = optimal_solve(cost_matrix, m_start, m_arrival)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Dec  8 2020 

command line - cbc /tmp/1c8f2b4876c341448b55f312a8cecf30-pulp.mps timeMode elapsed branch printingOptions all solution /tmp/1c8f2b4876c341448b55f312a8cecf30-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 526 RHS
At line 547 BOUNDS
At line 648 ENDATA
Problem MODEL has 20 rows, 100 columns and 200 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 9.73884e+06 - 0.00 seconds
Cgl0004I processed model has 20 rows, 100 columns (100 integer (0 of which binary)) and 200 elements
Cbc0012I Integer solution of 9738837.7 found by DiveCoefficient after 0 iterations and 0 nodes (0.05 seconds)
Cbc0001I Search completed - best objective 9738837.671875, took 0 iterations and 0 nodes (0.05 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 9.73884e

In [1227]:
def map_labels(img_array):
    map = {}
    size = img_array.size // 3
    


In [1228]:
def recolorized_image_array(img1_array, pixel_matrix2, optimal_plan, kmeans1, kmeans2, m_start):
        
    res = np.zeros_like(pixel_matrix2)
    unavailable_mass = [0 for l in range(len(m_start))]
    for j in range(kmeans2.cluster_centers_.shape[0]):
        counter = 0
        arg_cluster = np.where(kmeans2.labels_ == j)[0]
        for i in range(kmeans1.cluster_centers_.shape[0]):
            if optimal_plan[i][j] != 0.:
                target_positions = np.where(kmeans1.labels_ == i)[0][unavailable_mass[i]: unavailable_mass[i] + int(optimal_plan[i][j])]
                for pos in target_positions:
                    res[pos] = pixel_matrix2[arg_cluster[counter]]
                    counter += 1
                unavailable_mass[i] += int(optimal_plan[i][j])
    new_img = res.reshape(img1_array.shape)
    return new_img

In [1229]:
test_img = recolorized_image_array(img1_array, pixel_matrix2, optimal_plan, kmeans1, kmeans2, m_start)

In [1230]:
Image.fromarray(test_img).show()
Image.fromarray(test_img).save('result_' + str(number_cluster) + 'clust.jpg')